In [2]:
import tensorflow as tf
from keras import backend as K
import utils.utils_iterator as utils_iterator
import utils.utils_wv as utils_wv
import utils.attention_with_context as attention_layers
from keras.models import Model
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional, concatenate, Activation, Conv1D, Flatten, MaxPooling1D, Concatenate
import utils.utils_statistics as utils_statistics
import utils.utils_optimization as utils_optimization
from keras.callbacks import ModelCheckpoint
from keras import layers

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
checkpoint = ModelCheckpoint(
	    "models/best_model_kolev_3.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


sentence = Input(shape=(75,))
position = Input(shape=(75,))

embeddings = utils_wv.word2vec_embedding_layer()(sentence)

pos_embeddings = layers.Embedding(1,3, input_length=75, trainable=True)(position)
x = layers.concatenate([embeddings, pos_embeddings])
z = layers.Dropout(0.4)(x)
z = Bidirectional(LSTM(256, return_sequences=True))(z)
z = Bidirectional(LSTM(256, return_sequences=True))(z)
z = attention_layers.AttentionWithContext()(z)
z = Dense(10, activation="relu")(z)
z = layers.Dropout(0.5)(z)
z = Dense(1, activation="sigmoid")(z)


model = Model([sentence, position], z, name='sample_model')
model.summary()
model.compile(loss='binary_crossentropy',
		optimizer="adam",
		metrics=['accuracy'])
model.fit_generator(generator=utils_iterator.generate_data('../NLP_Preprocessing/data/clean_data/ontotext_train_labeled.txt', 512, input_size=75, with_positions=True), 
                    steps_per_epoch=71560//(512), 
                    validation_data=utils_iterator.generate_data('../NLP_Preprocessing/data/clean_data/ontotext_test_labeled.txt', 512, input_size=75, with_positions=True), 
                    validation_steps=17892//(512), workers=8, epochs=70, callbacks=callbacks_list)

utils_statistics.calculate_recall(model, 0.5)

NameError: name 'ModelCheckpoint' is not defined